In [1]:
import nltk
from nltk.corpus import names
import random
from nltk.classify import apply_features

#get data
names = ([(name, 'male') for name in names.words('male.txt')] +
[(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

In [2]:
def gender_features(word):
    return {'last_letter': word[-1]}

In [3]:
def gender_features2(name):
    features = {}
    features["firstletter"] = name[0].lower() #came with code; A
    features["lastletter"] = name[-1].lower() #came with code; B
    features["prefix"] = name[:3].lower() if len(name) > 4 else name[:2].lower() #gabby made; C
    features["suffix"] = name[-3:].lower() if len(name) > 4 else name[-2:].lower()#gabby made; D
    #features["final_consonants"] = name[-2:].lower() if name[-1].lower() not in ["a", "e", "i", "o", "u"] and name[-2].lower() not in ["a", "e", "i", "o", "u"] else False #gabby made; E
    features["final_vowels"] = name[-2:].lower() if name[-1].lower() in ["a", "e", "i", "o", "u"] and name[-2].lower() in ["a", "e", "i", "o", "u"] else False #gabby made; F
    features["name_length"] = len(name)
    #features["femme_endings"] = name[-4:].lower() if len(name) > 4 and name[-4].lower() in ["a", "e", "i", "o", "u"] and name[-3].lower() in ["a", "e", "i", "o", "u"] and name[-2].lower() == name[-3].lower() and name[-1].lower() in ["a", "e", "i", "o", "u", "y"] else False
    #name_len = len(name)
    #total_vowels = name.lower().count("a") + name.lower().count("e") + name.lower().count("i") + name.lower().count("o") + name.lower().count("u")
    #features["perc_consonants"] = int((name_len-total_vowels)/name_len)
    #features["perc_vowels"] = int(total_vowels/name_len)
    ### with the seven above, 0.829
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count(%s)" % letter] = name.lower().count(letter)
        features["has(%s)" % letter] = (letter in name.lower())
    return features

In [4]:
featuresets = [(gender_features2(n), g) for (n,g) in names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [5]:
print(nltk.classify.accuracy(classifier, train_set))

0.8619022031166039


In [6]:
print(nltk.classify.accuracy(classifier, test_set))

0.826


In [7]:
classifier.classify(gender_features2("Abney"))

'female'

In [8]:
classifier.show_most_informative_features(50)

Most Informative Features
            final_vowels = 'ia'           female : male   =     39.5 : 1.0
              lastletter = 'a'            female : male   =     34.6 : 1.0
              lastletter = 'k'              male : female =     30.6 : 1.0
                  suffix = 'tta'          female : male   =     25.8 : 1.0
                  suffix = 'ard'            male : female =     25.3 : 1.0
                  suffix = 'na'           female : male   =     18.6 : 1.0
                  suffix = 'old'            male : female =     18.2 : 1.0
                  suffix = 'nne'          female : male   =     18.1 : 1.0
                  suffix = 'vin'            male : female =     17.2 : 1.0
                  prefix = 'dor'          female : male   =     16.9 : 1.0
            final_vowels = 'io'             male : female =     15.9 : 1.0
              lastletter = 'f'              male : female =     15.9 : 1.0
                  suffix = 'ita'          female : male   =     14.5 : 1.0

In [9]:
train_names = names[1500:]
devtest_names = names[500:1500]
test_names = names[:500]

In [10]:
train_set = [(gender_features2(n), g) for (n,g) in train_names]
devtest_set = [(gender_features2(n), g) for (n,g) in devtest_names]
test_set = [(gender_features2(n), g) for (n,g) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.823


In [11]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features2(name))
    if guess != tag:
        errors.append( (tag, guess, name) )

In [12]:
print("Total errors: %d" % (len(errors)))
for (tag, guess, name) in sorted(errors):
    print('correct=%-8s guess=%-8s name=%-20s length=%-8s' % (tag, guess, name, len(name)))

Total errors: 177
correct=female   guess=male     name=Ambur                length=5       
correct=female   guess=male     name=Aphrodite            length=9       
correct=female   guess=male     name=Audy                 length=4       
correct=female   guess=male     name=Bell                 length=4       
correct=female   guess=male     name=Berty                length=5       
correct=female   guess=male     name=Blake                length=5       
correct=female   guess=male     name=Bonny                length=5       
correct=female   guess=male     name=Brandais             length=8       
correct=female   guess=male     name=Buffy                length=5       
correct=female   guess=male     name=Cameo                length=5       
correct=female   guess=male     name=Charis               length=6       
correct=female   guess=male     name=Cherish              length=7       
correct=female   guess=male     name=Chery                length=5       
correct=female   gue

In [13]:
def syllables(word): # I was making this to try and find a better way to look at words without importing a new library... meh.
    to_return = []
    consonants = list("bcdfghjklmnpqrstvwxzy")
    vowels = list("aeiouy")
    current_word = word.lower()
    while current_word != "":
        if current_word[0] in vowels and current_word[0] != "y":
            if current_word[1:3] == "we":
                to_return.append(current_word[:3])
                current_word = current_word[3:]
            else:
                if current_word[1] in consonants:
                    to_return.append(current_word[:2])
                    current_word = current_word[2:]
                elif current_word[2] in consonants:
                    to_return.append(current_word[:3])
                    current_word = current_word[3:]
        else:
            vowel_index = None
            for vowel in vowels:
                if vowel in current_word[:5] and vowel_index == None:
                    vowel_index = current_word.index(vowel)
            if len(current_word)-1 != vowel_index:
                if current_word[vowel_index+1] in vowels and current_word[vowel_index+1] != "i":
                    vowel_index += 1
                if current_word[vowel_index+1] == "s" and current_word[vowel_index+2] == "t":
                    to_return.append(current_word[:vowel_index+3])
                    current_word = current_word[vowel_index+3:]
                elif current_word[vowel_index+1] == "t" and current_word[vowel_index+2] == "h":
                    to_return.append(current_word[:vowel_index+3])
                    current_word = current_word[vowel_index+3:]
                else:
                    to_return.append(current_word[:vowel_index+2])
                    current_word = current_word[vowel_index+2:]
            else:
                to_return.append(current_word[:vowel_index+2])
                current_word = current_word[vowel_index+2:]
    return(to_return)